In [7]:
!which python
!pip list | grep pinecone


/opt/homebrew/Caskroom/miniconda/base/envs/mchatbot/bin/python
pinecone                  6.0.1
pinecone-plugin-interface 0.0.7


In [9]:
pip install -U langchain-community

  Using cached langchain_community-0.3.18-py3-none-any.whl.metadata (2.4 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
Using cached langchain_community-0.3.18-py3-none-any.whl (2.5 MB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from pinecone import Pinecone 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [11]:
import os
from dotenv import load_dotenv

# Load biến môi trường từ file .env
load_dotenv()

# Lấy giá trị từ biến môi trường
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT="us-east-1"


In [12]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [13]:
extracted_data = load_pdf("data/")

In [14]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [15]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7298


In [16]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
import sentence_transformers
print(sentence_transformers.__version__)


3.4.1


In [18]:
embeddings = download_hugging_face_embeddings()

/var/folders/1p/91jv3hhs53g1fjft5y02dvww0000gn/T/ipykernel_58354/1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [19]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [20]:
query_result = embeddings.embed_query("What is the capital of France?")
print("lenght", len(query_result))

lenght 384


In [12]:
#query_result

In [21]:
print(type(text_chunks[0]))  # Kiểm tra phần tử đầu tiên của text_chunks


<class 'langchain_core.documents.base.Document'>


In [22]:
 # Thêm import cho phiên bản mới

# Khởi tạo Pinecone với API key
pc = Pinecone(api_key=PINECONE_API_KEY)

# Kết nối đến index "medicalchat"
index = pc.Index("medicalchat")

# Batch size khi upsert
batch_size = 100

# Chuyển text thành vector embeddings đúng cách
'''vectors = [
    {
        "id": str(i),
        "values": embeddings.embed_query(text_chunks[i].page_content),  # Lấy nội dung văn bản
        "metadata": {"text": text_chunks[i].page_content}  # Lưu metadata
    }
    for i in range(len(text_chunks))
]



# Upsert theo batch
for i in range(0, len(vectors), batch_size):
    batch = vectors[i:i + batch_size]  # Chia batch nhỏ
    index.upsert(vectors=batch)  # Gửi lên Pinecone

print("Upsert hoàn thành!")  '''


'vectors = [\n   {\n       "id": str(i),\n       "values": embeddings.embed_query(text_chunks[i].page_content),  # Lấy nội dung văn bản\n       "metadata": {"text": text_chunks[i].page_content}  # Lưu metadata\n   }\n   for i in range(len(text_chunks))\n]\n\n\n\n# Upsert theo batch\nfor i in range(0, len(vectors), batch_size):\n   batch = vectors[i:i + batch_size]  # Chia batch nhỏ\n   index.upsert(vectors=batch)  # Gửi lên Pinecone\n\nprint("Upsert hoàn thành!")  '

In [23]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medicalchat")

In [24]:
query = "What are Allergies"
query_vector = embeddings.embed_query(query)
results = index.query(
    vector=query_vector,
    top_k=5,
    include_values=False,
    include_metadata=True
)

In [25]:
print(f"Kết quả tìm kiếm cho: '{query}'")
for match in results["matches"]:
    print(f"- Điểm số: {match['score']}")
    print(f"  Văn bản: {match['metadata']['text']}")
    print("-" * 20)

Kết quả tìm kiếm cho: 'What are Allergies'
- Điểm số: 0.749943137
  Văn bản: GALE ENCYCLOPEDIA OF MEDICINE 2116
Allergies
GEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 116
--------------------
- Điểm số: 0.739232242
  Văn bản: Mifflin, 1995.
Richard Robinson
Allergies
Definition
Allergies are abnormal reactions of the immune sys-
tem that occur in response to otherwise harmless sub-
stances.
GALE ENCYCLOPEDIA OF MEDICINE 2114
Allergies
GEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 114
--------------------
- Điểm số: 0.7338292
  Văn bản: An allergy is a type of immune reaction. Normally,
the immune system responds to foreign microorganisms
or particles by producing specific proteins called anti-
bodies. These antibodies are capable of binding to iden-
tifying molecules, or antigens, on the foreign particle.
This reaction between antibody and antigen sets off a
series of chemical reactions designed to protect the
--------------------
- Điểm số: 0.709889054
  Văn bản: Start Living. 2nd 

In [26]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [27]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [28]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [33]:
from langchain_pinecone import PineconeVectorStore


In [34]:
vectorstore = PineconeVectorStore(index_name="medicalchat", embedding=embeddings)

In [36]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})


In [37]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [38]:
user_input = input("Input Prompt: ")
result = qa({"query": user_input})
print("Response:", result["result"])


/var/folders/1p/91jv3hhs53g1fjft5y02dvww0000gn/T/ipykernel_58354/909192420.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": user_input})


Response: Skin fungus refers to a type of fungal infection that affects the skin. There are several types of fungi that can cause skin fungal infections, including Trichophyton, Epidermophyton, and Microsporum. These fungi can infect various parts of the body, including the feet (tinea pedis), nails (tinea unguium), and scalp (tinea capitis). Skin fungal infections can be mild or severe and may cause symptoms such as itching, redness, and flaking skin. Treatment typically involves antifungal medications applied topically or taken orally, depending on the severity of the infection.
